RAIN - Real & Artificial Intelligence for Neuroscience

## Create models
- This notebook will create and train Artificial Neural Networks to identify exploration using rodent and target position along with manually labeled data.

#### Requirements:

- A set of position files
- Labeled data for those position files (to train the model)

or

- Access to the example file **colabels.csv**, where we can find:
    - Position and labels for representative exploration events
    - Manual labels from 5 viewers (so far)

---
#### Load the necessary modules

In [1]:
from pathlib import Path
import rainstorm.modeling as rst

rainstorm.modeling successfully imported. GPU devices detected: []


---
#### 1. State your models project path
`base` : The path to the downloaded repository.

`models_folder` : The path to the folder containing the files you'll use to create the models.

In [2]:
base = Path(r'c:\Users\dhers\Desktop\Rainstorm')
models_folder = base / r'examples\models'

---
#### Create the colabels file

The Colabels file is a csv file that contains both mice positions, exploration target positions, and one or more sets of labels for the behavior you want to analyze.

If you want to train the models using your own colabels file, you can create it using the create_colabels function below. 

All you need is:
- A folder containing the positions of the mice
- A folder for each labeler, containing the labels for the behavior you want to analyze
- A list of the targets (stationary points) present on your videos


```python
path = r'path/to/colabels_folder' # The path to the directory containing the positions folder and labelers folders
labelers = ['labeler_A', 'labeler_B', 'etc']
targets = ['tgt_1', 'tgt_2', 'etc']

rst.create_colabels(path, labelers, targets)
```

##### There is a Colabels file available in the models folder which contains positions and labels for mice on a novel object recognition task. If you want to analyze the demo data, train the models using that colabels file.

---
#### 2.  Create the modeling.yaml file

The modeling.yaml file is a configuration file that contains all the parameters needed to create and train the models. It will be located in the models folder.

In [3]:
modeling = rst.create_modeling(models_folder)

modeling.yaml already exists: c:\Users\dhers\Desktop\Rainstorm\examples\models\modeling.yaml
Skipping creation.



It contains the following parameters:

`path` : Path to the models folder

`colabels` : The colabels file is used to store and organize positions and labels for model training
- colabels_path: Path to the colabels folder
- labelers: List of labelers on the colabels file (as found in the columns)
- target: Name of the target on the colabels file

`focus_distance`: Window of frames to consider around an exploration event

`bodyparts`: List of bodyparts used to train the model

`split`: Parameters for splitting the data into training, validation, and testing sets
- validation: Percentage of the data to use for validation
- test: Percentage of the data to use for testing

`RNN`: Set up the Recurrent Neural Network
- width: Defines the shape of the wide model
  - past : Number of past frames to include
  - future : Number of future frames to include
  - broad : Broaden the window by skipping some frames as we stray further from the present
- units: Number of neurons on each layer
- batch_size: Number of training samples the model processes before updating its weights
- epochs: Each epoch is a complete pass through the entire training dataset
- lr: Learning rate


---
#### 3. Before training a model, we need to prepare our training data
- First, we load the dataset from the colabels file and create one 'labels' column out of all the labelers.
- Next (optional, but recommended) we can erase the rows of the dataset that are too far away from exploration events.
- Finally, we split the dataset into training, testing and validation subsets.

In [4]:
# Prepare the data
dataset = rst.prepare_data(modeling)

# Focus on the rows near exploratory behaviour
dataset = rst.focus(modeling, dataset)

# Split the data
model_dict = rst.split_tr_ts_val(modeling, dataset)

# Save the split
rst.save_split(modeling, model_dict)

Focused around 'labels' events (10231 found)
Rows reduced: 167012 -> 38161
📊 Splitting data into training, validation, and test sets...
Training set:    26631 samples
Validation set:  5932 samples
Testing set:     5598 samples
Total samples:   38161
💾 Split data saved to: c:\Users\dhers\Desktop\Rainstorm\examples\models\splits\split_2025-06-09.h5


WindowsPath('c:/Users/dhers/Desktop/Rainstorm/examples/models/splits/split_2025-06-09.h5')

---
If you later want to load a previous split, run:

```python
saved_split = models_folder / 'splits/split_{example_date}.h5' # Select the split you want to rescue
model_dict = rst.load_split(saved_split)
```

---
We can see on the testing data that the exploratory events happen when the nose gets close to the object

In [5]:
rst.plot_example_data(model_dict['X_ts'], model_dict['y_ts'])

---
#### 4. With the training data ready, we can use TensorFlow to design your very first model
- It will look at the positions of one frame at a time, and try to decide if the mouse is exploring.
- If the decision is correct the architecture will be reinforced, else it will be corrected according to the learning rate.
- We will train it for some epochs (cycles through the whole dataset) and plot how the accuracy and loss evolve.
- Also, we will be validating the training using the validation split, which contains frames that were not used for training.

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
import datetime
time = datetime.datetime.now()

# Build a simple neural network
model_simple = tf.keras.Sequential([
    
    # Input layer
    Input(shape=(model_dict['X_tr'].shape[1],)), 

    # Hidden layers
    Dense(32, activation='relu'),
    Dense(24, activation='relu'),
    Dense(16, activation='relu'),
    Dense(12, activation='relu'),
    Dense(8, activation='relu'),
    
    # Output layer
    Dense(1, activation='sigmoid')
])

# Compile the model
model_simple.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001), 
                   loss='binary_crossentropy', metrics=['accuracy'])

model_simple.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                416       
                                                                 
 dense_1 (Dense)             (None, 24)                792       
                                                                 
 dense_2 (Dense)             (None, 16)                400       
                                                                 
 dense_3 (Dense)             (None, 12)                204       
                                                                 
 dense_4 (Dense)             (None, 8)                 104       
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,925
Trainable params: 1,925
Non-trainabl

#### Train the model
Each ``epoch`` is a complete pass through the entire training dataset, while the ``batch_size`` is the number of training samples the model processes before updating its weights

In [7]:
history_simple = model_simple.fit(model_dict['X_tr'], model_dict['y_tr'],
                                  epochs=10, batch_size=128,
                                  validation_data=(model_dict['X_val'], model_dict['y_val']))

Epoch 1/10
209/209 [==============================] - 1s 2ms/step - loss: 0.6928 - accuracy: 0.5713 - val_loss: 0.6880 - val_accuracy: 0.6719
Epoch 2/10
209/209 [==============================] - 0s 2ms/step - loss: 0.6800 - accuracy: 0.6911 - val_loss: 0.6444 - val_accuracy: 0.6856
Epoch 3/10
209/209 [==============================] - 0s 1ms/step - loss: 0.4973 - accuracy: 0.6792 - val_loss: 0.4402 - val_accuracy: 0.6773
Epoch 4/10
209/209 [==============================] - 0s 1ms/step - loss: 0.3589 - accuracy: 0.7116 - val_loss: 0.3770 - val_accuracy: 0.7045
Epoch 5/10
209/209 [==============================] - 0s 1ms/step - loss: 0.2940 - accuracy: 0.7612 - val_loss: 0.3298 - val_accuracy: 0.7308
Epoch 6/10
209/209 [==============================] - 0s 2ms/step - loss: 0.2519 - accuracy: 0.7842 - val_loss: 0.3052 - val_accuracy: 0.7370
Epoch 7/10
209/209 [==============================] - 0s 1ms/step - loss: 0.2319 - accuracy: 0.7905 - val_loss: 0.2984 - val_accuracy: 0.7456
Epoch 

#### Plot the training and validation loss

In [8]:
rst.plot_history(history_simple, "Simple")

#### Calculate accuracy and precision of the model

In [9]:
y_pred_simple = model_simple.predict(model_dict['X_ts'])
metrics_simple = rst.evaluate(y_pred_simple, model_dict['y_ts'], show_report = True)

175/175 [==============================] - 0s 627us/step

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.95      0.92      0.94      4370
           1       0.74      0.85      0.79      1228

    accuracy                           0.90      5598
   macro avg       0.85      0.88      0.86      5598
weighted avg       0.91      0.90      0.90      5598

-----------------------------
Accuracy: 0.9014
Precision: 0.7411
Recall: 0.8461
F1_Score: 0.7901
MSE: 0.0533
MAE: 0.1265
R2: 0.6580
-----------------------------



#### And finally, save the model

In [10]:
model_name = f'{time.date()}_simple'
rst.save_model(modeling, model_simple, model_name)

Model '2025-06-09_simple' saved to: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-09_simple.keras


---
#### 5. Now that we have a simple model trained, we can start building more complex models

To make our artificial networks as real as possible, we can let them see a sequence of frames to decide if the mouse is exploring
- Our build_RNN function will use Bidirectional LSTM layers that allow the model to take into account the temporal sequence of frames
- It also implements early stopping and learning rate scheduler mechanisms that will prevent the model from overfitting

We can control the RNN model by changing the following parameters on the modeling.yaml file:

`units` : The number of neurons on each layer of the LSTM model

`batch_size` : The number of training samples the model processes before updating its weights

`lr` : The learning rate of the model

`epochs` : The number of times the model will be trained on the entire training dataset

`past` & `future` : If you use a LSTM model, you can set the window size by saying how many frames into the past and how many into the future you want to see.

`broad` : Once you have your window size, we can broaden the window by skipping some frames as it strays further from the present.

In [13]:
model_wide = rst.build_RNN(modeling, model_dict)

Model: "ModularBidirectionalRNN"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_sequence (InputLayer)    [(None, 7, 12)]      0           []                               
                                                                                                  
 bilstm_0 (Bidirectional)       (None, 7, 32)        3712        ['input_sequence[0][0]']         
                                                                                                  
 bn_0 (BatchNormalization)      (None, 7, 32)        128         ['bilstm_0[0][0]']               
                                                                                                  
 dropout_0 (Dropout)            (None, 7, 32)        0           ['bn_0[0][0]']                   
                                                                            

#### Train the model

In [15]:
model_wide_name = f'{time.date()}_wide'
history_wide = rst.train_RNN(modeling, model_wide, model_dict, model_wide_name)

🚀 Starting training for model: 2025-06-09_wide
Epoch 1/100
417/417 [==============================] - 23s 29ms/step - loss: 1.0782 - accuracy: 0.4400 - val_loss: 0.6687 - val_accuracy: 0.5531 - lr: 1.0000e-05
Epoch 2/100
417/417 [==============================] - 10s 23ms/step - loss: 0.7623 - accuracy: 0.5453 - val_loss: 0.4944 - val_accuracy: 0.6721 - lr: 1.9000e-05
Epoch 3/100
417/417 [==============================] - 10s 25ms/step - loss: 0.5671 - accuracy: 0.6311 - val_loss: 0.4011 - val_accuracy: 0.7218 - lr: 2.8000e-05
Epoch 4/100
417/417 [==============================] - 10s 25ms/step - loss: 0.4589 - accuracy: 0.6909 - val_loss: 0.3659 - val_accuracy: 0.7399 - lr: 3.7000e-05
Epoch 5/100
417/417 [==============================] - 11s 26ms/step - loss: 0.3908 - accuracy: 0.7233 - val_loss: 0.3556 - val_accuracy: 0.7449 - lr: 4.6000e-05
Epoch 6/100
417/417 [==============================] - 11s 26ms/step - loss: 0.3514 - accuracy: 0.7402 - val_loss: 0.3420 - val_accuracy: 0.747

#### Plot the training and validation loss

In [16]:
rst.plot_history(history_wide, model_wide_name)

#### Calculate accuracy and precision of the model

In [20]:
y_pred_wide = model_wide.predict(model_dict['X_ts_wide'])
metrics_wide = rst.evaluate(y_pred_wide, model_dict['y_ts'], show_report=True)

175/175 [==============================] - 1s 7ms/step

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      4370
           1       0.71      0.89      0.79      1228

    accuracy                           0.90      5598
   macro avg       0.84      0.89      0.86      5598
weighted avg       0.91      0.90      0.90      5598

-----------------------------
Accuracy: 0.8953
Precision: 0.7068
Recall: 0.8933
F1_Score: 0.7892
MSE: 0.0584
MAE: 0.1103
R2: 0.6247
-----------------------------



#### Save the wide model

In [21]:
rst.save_model(modeling, model_wide, model_wide_name)

Model '2025-06-09_wide' saved to: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-09_wide.keras


---
#### 6. Finally, compare the trained models

- Since we trained using the training dataset, and validated using the validation dataset... we test each model using the testing dataset.

In [23]:
# Print the model results
print("Evaluate model vs testing data")
print(f"{metrics_simple} -> simple")
print(f"{metrics_wide} -> wide")

Evaluate model vs testing data
{'Accuracy': 0.9013933547695605, 'Precision': 0.7410841654778887, 'Recall': 0.8460912052117264, 'F1_Score': 0.7901140684410647, 'MSE': 0.05326417348115438, 'MAE': 0.1265376053060921, 'R2': 0.6580055819216821} -> simple
{'Accuracy': 0.8953197570560915, 'Precision': 0.7068298969072165, 'Recall': 0.8933224755700325, 'F1_Score': 0.7892086330935252, 'MSE': 0.058444908706036275, 'MAE': 0.11034273756987231, 'R2': 0.624741524438875} -> wide


---
---
#### Our trained models are stored safely in our repository, with today's date.
We can:
- Continue on this notebook and evaluate the trained models.
- Skip the evaluation and go use the models on our data in `3b-Automatic_analysis`.
---


## Evaluate models

I see you've decided to continue on this notebook! You wont regret it.

One may think that the evaluation we did on the testing set is enough, and in many cases it is. However, for our purpose of finding a model that resembles the labeling of an expert, It's better to compare the performance of the model against all the manually labeled data we have.

In [2]:
evaluation_dict = rst.build_evaluation_dict(modeling)

---
#### 7. Calculate a good reference labeler
Since we want to compare the models and the labelers, we need to create a reference labeler.

This reference could be the mean of all the labelers, but then the labelers would have an unfair advantage.

To avoid this, we choose to simultaneously create a chimera labeler and a leave-one-out-mean:
- The chimera is created by randomly selecting a labeler on each row of the data.
- The leave-one-out-mean is created by averaging the remaining labelers.

This way, we can compare the chimera to the leave-one-out-mean knowing that they are independent.

In [3]:
chimera_dict = rst.create_chimera_and_loo_mean(evaluation_dict['manual_labels'], seed=42)

---
#### 8. Load the models & use them to label exploration on all the available data

In [4]:
# List the models you want to evaluate
model_paths = {
    'example_simple': models_folder / 'trained_models/example_simple.keras',
    'example_wide': models_folder / 'trained_models/example_wide.keras',
    f'{time.date()}_simple': models_folder / f'trained_models/{time.date()}_simple.keras',
    f'{time.date()}_wide': models_folder / f'trained_models/{time.date()}_wide.keras',
    # Add more models as needed...
    }

In [5]:
models_dict = rst.build_and_run_models(modeling, model_paths, evaluation_dict['position'])

Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\example_simple.keras
5220/5220 [==============================] - 4s 668us/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\example_wide.keras
5220/5220 [==============================] - 15s 3ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-09_simple.keras
5220/5220 [==============================] - 3s 535us/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-09_wide.keras
5220/5220 [==============================] - 38s 7ms/step


In [6]:
evaluation_dict.update(chimera_dict)
evaluation_dict.update(models_dict)
evaluation_dict = {k: v for k, v in evaluation_dict.items() if k not in {'position', 'manual_labels'}}
print(evaluation_dict.keys())  # Check the keys to confirm the additions

dict_keys(['Labeler_A', 'Labeler_B', 'Labeler_C', 'Labeler_D', 'Labeler_E', 'geometric', 'chimera', 'loo_mean', 'model_example_simple', 'model_example_wide', 'model_2025-06-09_simple', 'model_2025-06-09_wide'])


---
#### 9. With all the labels organized, we can evaulate the performance of each

In [7]:
for name, pred in evaluation_dict.items():
    metrics = rst.evaluate(pred, evaluation_dict['loo_mean'])
    print(f"{metrics} -> {name}")

{'Accuracy': 0.9769118386702752, 'Precision': 0.7069446597426755, 'Recall': 0.991844280121792, 'F1_Score': 0.825504570549371, 'MSE': 0.01142170921849927, 'MAE': 0.02253281201350801, 'R2': 0.7654496770367311} -> Labeler_A
{'Accuracy': 0.9825940650971188, 'Precision': 0.9736406085253804, 'Recall': 0.702914310569813, 'F1_Score': 0.8164193242816545, 'MSE': 0.015732776686705148, 'MAE': 0.02684387948171389, 'R2': 0.6769198215098198} -> Labeler_B
{'Accuracy': 0.9648408497592987, 'Precision': 0.6118890534536152, 'Recall': 0.98836450630709, 'F1_Score': 0.7558419958419959, 'MSE': 0.018322410964481593, 'MAE': 0.029433513759490337, 'R2': 0.623740429127335} -> Labeler_C
{'Accuracy': 0.9901504083538908, 'Precision': 0.8752609084584038, 'Recall': 0.9575902566333189, 'F1_Score': 0.9145765176299527, 'MSE': 0.008550658036548272, 'MAE': 0.019661760831557015, 'R2': 0.8244081016550004} -> Labeler_D
{'Accuracy': 0.9927729743970494, 'Precision': 0.9476633419253614, 'Recall': 0.9195302305350153, 'F1_Score': 0

We can visualize the similarity between labelers using a cosine similarity plot

In [ ]:
rst.plot_cosine_sim(evaluation_dict)

And finally, run a PCA (Principal Components Analysis) to see how much each labeler resembles eachother and the mean

In [ ]:
rst.plot_PCA(evaluation_dict)

Also, we can see both the models and the labelers performance on an example video

In [10]:
example_folder_path = base / r'examples\colabeled_video'

labelers_example = {
    "labeler_A": "Example_labeler_A.csv",
    "labeler_B": "Example_labeler_B.csv",
    "labeler_C": "Example_labeler_C.csv",
    "labeler_D": "Example_labeler_D.csv",
    "labeler_E": "Example_labeler_E.csv"
}

rst.plot_performance_on_video(example_folder_path, model_paths, labelers_example, fps = 25, 
                              bodyparts = ['nose', 'left_ear', 'right_ear', 'head', 'neck', 'body'], 
                              targets = ['obj_1', 'obj_2'], plot_tgt = "obj_2")

Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\example_simple.keras
469/469 [==============================] - 0s 588us/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\example_wide.keras
469/469 [==============================] - 3s 2ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-09_simple.keras
469/469 [==============================] - 1s 1ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-09_wide.keras
469/469 [==============================] - 5s 5ms/step


---
---
#### Once we get to this point, we should have selected our favorite model.
We can move on to the next notebook, `3b-Automatic_analysis`, and use the chosen model to label our position files.

---
RAINSTORM - Created on Dec 12, 2023 - @author: Santiago D'hers